In [7]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
    sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [8]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [9]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # Initialize (all 0) a second array with the value function for next time step
        V_next = np.zeros(env.nS)
        for s, actions in env.P.items():
            for a in actions.keys():
                for prob, next_state, reward, done in env.P[s][a]:
                    # IRL, Eq. (4.5)
                    V_next[s] += policy[s, a] * prob * (reward + discount_factor*V[next_state])
        # Stop evaluation once value funtion change insignificant
        if np.all(np.abs(V_next - V) < theta):
            break
        else:
            # Update current value function by next-time-step value function
            V = V_next
    return V

In [10]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    
    def one_step_lookahead(s, V):
        '''
            One step lookahead, Eq. (4.9)
        '''
        action_values = np.zeros(env.nA)
        for a in env.P[s].keys():
            v = 0
            for prob, next_state, reward, done in env.P[s][a]:
                v += prob*(reward + discount_factor*V[next_state])
            action_values[a] = v
        return action_values
    
    # Initialization
    # Start with random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        
        #######################
        # Policy evaluation
        #######################
        V = policy_eval(policy=policy, env=env)
        
        #######################
        # Policy improvement
        #######################
        policy_stable = True
        for s in range(env.nS):
            # choose current action by acting greedily on current policy
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            # act greedily wrt to current value function
            best_a = np.argmax(action_values)
            policy[s] = np.eye(env.nA)[best_a]
            # if policy does not change anymore, it converged to optimal
            if best_a != chosen_a:
                policy_stable = False
        if policy_stable:
            return (policy, V)

In [11]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [12]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)